In [ ]:
import mrcnn
import mrcnn.config
import mrcnn.model
import mrcnn.visualize
import cv2
import os
import numpy as np

In [ ]:
# load the class label names from disk, one label per line
# CLASS_NAMES = open("coco_labels.txt").read().strip().split("\n")

CLASS_NAMES = ['BG', 'armchair', 'bed', 'door1', 'door2', 'sink1', 'sink2', 'sink3', 'sink4', 'sofa1', 'sofa2', 'table1', 'table2', 'table3', 'tub', 'window1', 'window2']

In [ ]:
class SimpleConfig(mrcnn.config.Config):
    NAME = 'furnitures'  # Override in sub-classes

    GPU_COUNT = 1

    IMAGES_PER_GPU = 1

    STEPS_PER_EPOCH = 1

    VALIDATION_STEPS = 1

    BACKBONE = "resnet101"

    # Number of classification classes (including background)
    NUM_CLASSES = 1 + 16 # Override in sub-classes

    IMAGE_RESIZE_MODE = "square"
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    IMAGE_CHANNEL_COUNT = 1

    # Image mean (RGB)
    MEAN_PIXEL = np.array([128.0])

    # Learning rate and momentum
    # The Mask RCNN paper uses lr=0.02, but on TensorFlow it causes
    # weights to explode. Likely due to differences in optimizer
    # implementation.
    LEARNING_RATE = 0.001
    LEARNING_MOMENTUM = 0.9

    # Weight decay regularization
    WEIGHT_DECAY = 0.0001

    # Loss weights for more precise optimization.
    # Can be used for R-CNN training setup.
    LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 1.
    }

In [ ]:
# Initialize the Mask R-CNN model for inference and then load the weights.
# This step builds the Keras model architecture.
model = mrcnn.model.MaskRCNN(mode="inference", 
                             config=SimpleConfig(),
                             model_dir=os.getcwd())

In [ ]:
# Load the weights into the model.
# Download the mask_rcnn_coco.h5 file from this link: https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
model.load_weights(filepath="./logs/furnitures/furniture_segment.h5", by_name=True)

In [ ]:
# load the input image, convert it from BGR to RGB channel
image = cv2.imread("../dataset/train/floor_image_22.tiff", cv2.IMREAD_GRAYSCALE)
image = np.expand_dims(image, axis=-1)

# Perform a forward pass of the network to obtain the results
r = model.detect([image], verbose=0)

# Get the results for the first image.
r = r[0]

print(image.shape, r['masks'].shape, r['class_ids'].shape, r['rois'].shape)

In [ ]:
# Visualize the detected objects.
mrcnn.visualize.display_instances(image=image, 
                                  boxes=r['rois'], 
                                  masks=r['masks'], 
                                  class_ids=r['class_ids'], 
                                  class_names=CLASS_NAMES, 
                                  scores=r['scores'])